<a href="https://colab.research.google.com/github/malinphy/q_17/blob/main/patent_data_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os 
import json

In [3]:
biotech_patents_dir = 'drive/MyDrive/transfer/patent/vec_vectorized_biotech_patents_transformed_ols.json'
uspto_patents_dir = 'drive/MyDrive/transfer/patent/vec_vectorized_uspto_patents_transformed_ols.json'
nih_patents_dir = 'drive/MyDrive/transfer/patent/patent_raw/nih/nih_patents_ada_embeddings.json'

In [4]:
# with open(nih_patents_dir, 'r') as f:
#     data = json.load(f)

In [5]:
class patent_data:
    def __init__(self, dir):
        self.dir = dir

    def ab_title_bringer(self):
        with open (self.dir, 'r') as f:
            data = json.load(f)

        title = [data[i]['title'] for i in range(len(data))]
        abstract = [data[i]['abstract'] for i in range(len(data))]
        urls = [data[i]['url'] for i in range(len(data))]

        return title, abstract, urls

In [6]:
biotech_data = patent_data(biotech_patents_dir)
biotech_title, biotech_abstract,biotech_urls = biotech_data.ab_title_bringer()

uspto_data = patent_data(uspto_patents_dir)
uspto_title, uspto_abstract, biotech_urls = uspto_data.ab_title_bringer()

nih_data = patent_data(nih_patents_dir)
nih_title, nih_abstract, biotech_urls = nih_data.ab_title_bringer()

In [7]:
def normalizer(string):
    return string.lower().rstrip().lstrip()

In [8]:
normalized_title_biotech = [normalizer(i) for i in biotech_title]
normalized_abstract_biotech = [normalizer(i) for i in biotech_abstract]

normalized_title_uspto = [normalizer(i) for i in uspto_title]
normalized_abstract_uspto = [normalizer(i) for i in uspto_abstract]

nih_title_biotech = [normalizer(i) for i in nih_title]
nih_abstract_biotech = [normalizer(i) for i in nih_abstract]

In [9]:
print('Title BIOTECH PATENTS :',len(normalized_title_biotech))
print('Unique titles BIOTECH PATENTS :',len(set(normalized_title_biotech)))
print('Title USPTO patents :',len(normalized_title_uspto))
print('Unique titles USPTO patents :',len(set(normalized_title_uspto)))
print('Title NIH PATENTS :',len(nih_title_biotech))
print('Unique titles NIH PATENTS :',len(set(nih_title_biotech)))

Title BIOTECH PATENTS : 484
Unique titles BIOTECH PATENTS : 228
Title USPTO patents : 4
Unique titles USPTO patents : 4
Title NIH PATENTS : 3243
Unique titles NIH PATENTS : 2842


In [10]:
print('Title BIOTECH PATENTS :',len(normalized_abstract_biotech))
print('Unique titles BIOTECH PATENTS :',len(set(normalized_abstract_biotech)))
print('Title USPTO patents :',len(normalized_abstract_uspto))
print('Unique titles USPTO patents :',len(set(normalized_abstract_uspto)))
print('Title NIH PATENTS :',len(nih_abstract_biotech))
print('Unique titles NIH PATENTS :',len(set(nih_abstract_biotech)))

Title BIOTECH PATENTS : 484
Unique titles BIOTECH PATENTS : 231
Title USPTO patents : 4
Unique titles USPTO patents : 4
Title NIH PATENTS : 3243
Unique titles NIH PATENTS : 2865


In [12]:
import collections

original = ['A', 'B', 'A', 'B', 'C', 'A','C']
dup = [item for item, count in collections.Counter(original).items() if count > 1]
for i in dup:
    print("index of", i, "=", original.index(i))

index of A = 0
index of B = 1
index of C = 4
